# "Concevez une application au service de la santé publique"
_Analysis Notebook_

In [12]:
from google.colab import files, drive
drive.mount('/gdrive')
%cd /gdrive

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).
/gdrive


In [0]:
# May need to reboot the environment
!pip install -r "My Drive/--DATA SCIENCE/PROJET3/requirements.txt"

In [0]:
import simplejson as json
import pandas as pd
import os

In [0]:
# get list of dtype from json
f = open('My Drive/--DATA SCIENCE/PROJET3/d_types.json', 'r')
d_types = json.loads(f.read())

In [0]:
# Import cleaned dataset
dftest = pd.read_csv('My Drive/--DATA SCIENCE/PROJET3/OFF_Cleaned.csv',
                     dtype=d_types, low_memory=False, encoding ='utf-8')

In [17]:
dftest.shape

(560201, 105)

In [18]:
dftest.columns

Index(['Unnamed: 0', 'code', 'code_val', 'url', 'creator', 'created_datetime',
       'main_category_en', 'main_category', 'categories_en', 'categories',
       ...
       'caffeine_100g', 'fruits-vegetables-nuts_100g',
       'fruits-vegetables-nuts-estimate_100g', 'cocoa_100g',
       'carbon-footprint_100g', 'energy-from-fat-kj_100g', 'energy-kcal_100g',
       'energy-kj_100g', 'carbon-footprint-from-meat-or-fish_100g',
       'phylloquinone_100g'],
      dtype='object', length=105)